## Sentiment Analysis Challenge

Imports.

In [233]:
import csv
import math

Set input variables.

In [234]:
train = True
data_file = 'data/train.tsv'
validation_percent = 0.3

Import the data from provided file, only including valid rows with both labels and data.

In [235]:
labels = []
inputs = []
with open('data/train.tsv', encoding='utf-8') as data:
  reader = csv.reader(data, delimiter='\t')
  for row in reader:
    if len(row) == 2:
        labels.append(row[0])
        inputs.append(row[1])

In [236]:
num_train_samples = int(len(labels) * (1-validation_percent))
train_labels = labels[:num_train_samples]
train_inputs = inputs[:num_train_samples]
val_labels = labels[num_train_samples:]
val_inputs = inputs[num_train_samples:]

In [237]:
all_words = []
positive_words = []
negative_words = []
for idx, document in enumerate(train_inputs):
    all_words.extend(document.split())
    positive_words.extend(document.split()) if int(labels[idx]) else negative_words.extend(document.split())

In [238]:
def calcPositive(word):
    return math.log((positive_words.count(word) + 1) / len(positive_words))
def calcNegative(word):
    return math.log((negative_words.count(word) + 1) / len(negative_words))

In [239]:
def predict(document):
    positive_sum = 0
    negative_sum = 0
    for word in document.split():
        positive_sum += calcPositive(word)
        negative_sum += calcNegative(word)
    return 1 if positive_sum > negative_sum else 0

In [240]:
correct = 0
for idx, doc in enumerate(val_inputs[:200]):
    print(idx)
    if predict(doc) == int(val_labels[idx]):
        correct += 1
print("Validation Accuracy:", correct / 200)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
Validation Accuracy: 0.715
